In [5]:
import tweepy
#Used for authentication protocal(OAuth) of Twitter API
from tweepy import OAuthHandler
#This will allow us to create a stream from twitter
from tweepy import Stream
#Will allow us to get the data from the port number
from tweepy.streaming import StreamListener
#It is used to create socket/port on the local machine 
import socket
#json is used because twitter data comes in json format 
import json
from decouple import config

In [6]:
consumer_key = config('tweeter_consumer_key')
consumer_secret = config('tweeter_consumer_secret')
access_token = config('tweeter_access_token')
access_secret = config('tweeter_access_secret')

class TweetsListener(StreamListener):
    def __init__(self,csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:
            msg = json.loads(data)
            print(msg["text"].encode("utf-8"))
            self.client_socket.send(msg["text"].encode("utf-8"))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    def on_error(self, status):
        print(status)
        return True

In [7]:
def sendData(c_socket):
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token,access_secret)
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=["BillGates"])

In [20]:

if __name__ == "__main__":
    s = socket.socket() #create a socket object
    host = "127.0.0.1" #Get local machine name
    port = 9091 #Reserve a port for my service
    s.bind((host, port)) #Bind to the port
    print("Listening on port: %s" % str(port))
    s.listen(5)  #wait for the client connection
    c, addr = s.accept() #establishing connection with client
    print("Received request from: " + str(addr)) 
    sendData(c)

OSError: [Errno 98] Address already in use